# 771948_A23_T3A - Group Work Assignment #
# Task 1 - Numerical and categorical feature classification problem #
## Assignment by Chris Mintz 202369825 and Antonia Agunbiade [STUDENT ID HERE] ##

### Task 1 - Load and preprocess the dataset for the classification problem (handle missing data, convert categorical features to numerical features) ###

### Task 2 - Build a classifier for the classification problem using one of the specified models (logistic regression, decision trees, random forests, or artificial neural networks) ###

# Task 3 - Fine tune the selected model using appropriate techniques like hyperparameter tuning, cross-validation, etc.

# Task 4 - Visualise the dataset and the model's results, where applicable like feature importance, confusion matrix, etc

# Task 5 - Report the final performance of the selected model using the appropriate performance metrics like accuracy, F1-score, etc